# Thinkful 28.4 - Supervised Neural Networks

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

We're ready to build our first neural network. We will have multiple features we feed into our model, each of which will go through a set of perceptron models to arrive at a response which will be trained to our output.

Like many models we've covered, this can be used as both a regression or classification model.

First, we need to load our dataset. For this example we'll use The Museum of Modern Art in New York's [public dataset](https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv) on their collection.

In [2]:
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [3]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

We'll also do a bit of data processing and cleaning, selecting columns of interest and converting URL's to booleans indicating whether they are present.

In [4]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [5]:
artworks.shape

(108603, 10)

In [6]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


## Building a Model

Now, let's see if we can use multi-layer perceptron modeling (or "MLP") to see if we can classify the department a piece should go into using everything but the department name.

Before we import MLP from SKLearn and establish the model we first have to ensure correct typing for our data and do some other cleaning.

In [7]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

The `DateAcquired` column is an object. Let's transform that to a datetime object and add a feature for just the year the artwork was acquired.

In [8]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

Great. Let's do some more miscellaneous cleaning.

In [9]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [15]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(1000,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [16]:
mlp.score(X, Y)

0.7111774076222572

In [17]:
Y.value_counts()/len(Y)

Drawings & Prints        0.622883
Photography              0.226228
Architecture & Design    0.112842
Painting & Sculpture     0.033636
Media and Performance    0.004411
Name: Department, dtype: float64

In [13]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, Y, cv=5)

array([0.45883864, 0.60429944, 0.56337693, 0.54309361, 0.42465037])

Now we got a lot of information from all of this. Firstly we can see that the model seems to overfit, though there is still so remaining performance when validated with cross validation. This is a feature of neural networks that aren't given enough data for the number of features present. _Neural networks, in general, like_ a lot _of data_. You may also have noticed something also about neural networks: _they can take a_ long _time to run_. Try increasing the layer size by adding a zero. Feel free to interrupt the kernel if you don't have time...

Also note that we created bools for artist's name but left them out. Both of the above points are the reason for that. It would take much longer to run and it would be much more prone to overfitting.

## Model parameters

Now, before we move on and let you loose with some tasks to work on the model, let's go over the parameters.

We included one parameter: hidden layer size. Remember in the previous lesson, when we talked about layers in a neural network. This tells us how many and how big to make our layers. Pass in a tuple that specifies each layer's size. Our network is 1000 neurons wide and one layer. (100, 4, ) would create a network with two layers, one 100 wide and the other 4.

How many layers to include is determined by two things: computational resources and cross validation searching for convergence. It's generally less than the number of input variables you have.

You can also set an alpha. Neural networks like this use a regularization parameter that penalizes large coefficients just like we discussed in the advanced regression section. Alpha scales that penalty.

Lastly, we'll discuss the activation function. The activation function determines whether the output from an individual perceptron is binary or continuous. By default this is a 'relu', or 'rectified linear unit function' function. In the exercise we went through earlier we used this binary function, but we discussed the _sigmoid_ as a reasonable alternative. The _sigmoid_ (called 'logistic' by SKLearn because it's a 'logistic sigmoid function') allows for continuous variables between 0 and 1, which allows for a more nuanced model. It does come at the cost of increased computational complexity.

If you want to learn more about these, study [activation functions](https://en.wikipedia.org/wiki/Activation_function) and [multilayer perceptrons](https://en.wikipedia.org/wiki/Multilayer_perceptron). The [Deep Learning](http://www.deeplearningbook.org/) book referenced earlier goes into great detail on the linear algebra involved.

You could also just test the models with cross validation. Unless neural networks are your specialty cross validation should be sufficient.

For the other parameters and their defaults, check out the [MLPClassifier documentaiton](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).

## Drill: Playing with layers

Now it's your turn. Using the space below, experiment with different hidden layer structures. You can try this on a subset of the data to improve runtime. See how things vary. See what seems to matter the most. Feel free to manipulate other parameters as well. It may also be beneficial to do some real feature selection work...

In [18]:
# Your code here. Experiment with hidden layers to build your own model.

#reduce size of dataset to improve speed
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size = 0.2)
X_train.head()

,URL,ThumbnailURL,Height (cm),Width (cm),YearAcquired,Gender_(),Gender_(Female),Gender_(Male),Gender_(male),Gender_\(multiple_persons\),...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
90744,True,True,25.7176,7.62,2007,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61814,True,True,30.0000,31.70,1968,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
31083,False,False,21.5000,27.20,1974,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
33515,True,True,29.2000,25.40,1995,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27587,False,False,18.2000,19.20,1964,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Try modeling with defaulthidden layer settings
mlp2 = MLPClassifier(hidden_layer_sizes=())
mlp2.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [20]:
mlp2.score(X_train, y_train)

0.7203959484346225

In [24]:
# 5-fold CV
scores = cross_val_score(mlp2, X_train, y_train, cv=5)
print('Scores: ', scores)

print('\nAccuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

Scores:  [0.71129515 0.71178637 0.70833333 0.71056873 0.70796868]

Accuracy: 0.71 (+/- 0.00)


Reducing the size of the dataset by 80% produced approximately the same score as that of the entire dataset, but significantly reduced its overfitting, as indicated by the standard deviation of its 5-fold cross-validation.

### Sigmoid Logistic Activation Function

In [25]:
mlp_sig = MLPClassifier(activation='logistic')
mlp_sig.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [26]:
mlp_sig.score(X_train, y_train)

0.7588858195211786

In [27]:
# 5-fold CV
scores = cross_val_score(mlp_sig, X_train, y_train, cv=5)
print('Scores: ', scores)

print('\nAccuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

C:\Users\Amichai\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Scores:  [0.73890039 0.72099448 0.73687845 0.75500806 0.76163058]

Accuracy: 0.74 (+/- 0.03)


The logistic/sigmoid activation improved the score even more, and the standard deviation of cross-validation scores indicates a relatively low rate of overfitting.

### Alpha Parameter

In [28]:
mlp_sig2 = MLPClassifier(activation='logistic', alpha = 0.1)
mlp_sig2.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=0.1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [29]:
mlp_sig2.score(X_train, y_train)

0.7218692449355433

In [30]:
# 5-fold CV
scores = cross_val_score(mlp_sig2, X_train, y_train, cv=5)
print('Scores: ', scores)

print('\nAccuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

Scores:  [0.71957672 0.69498158 0.71224678 0.71793691 0.7192538 ]

Accuracy: 0.71 (+/- 0.02)


In [35]:
mlp_sig3 = MLPClassifier(activation='logistic', alpha = 1e-6)
mlp_sig3.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=1e-06, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [36]:
mlp_sig3.score(X_train, y_train)

0.7715469613259669

In [37]:
# 5-fold CV
scores = cross_val_score(mlp_sig3, X_train, y_train, cv=5)
print('Scores: ', scores)

print('\nAccuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

Scores:  [0.74166092 0.73526703 0.75161142 0.73681787 0.74044219]

Accuracy: 0.74 (+/- 0.01)


In [38]:
mlp_sig4 = MLPClassifier(activation='logistic', alpha = 1e-9)
mlp_sig4.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=1e-09, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [39]:
mlp_sig4.score(X_train, y_train)

0.7498618784530386

In [40]:
# 5-fold CV
scores = cross_val_score(mlp_sig4, X_train, y_train, cv=5)
print('Scores: ', scores)

print('\nAccuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

Scores:  [0.73613987 0.73135359 0.7325046  0.74902141 0.76531552]

Accuracy: 0.74 (+/- 0.03)


The optimal `alpha` value is 1e-6 or 0.000001. It provides the highest overall accuracy score of the parameters tested and also has the lowest rate of overfitting, based on the standard deviation from the cross-validation.

### Number of Layers

In [41]:
# Establish and fit the model, with logistic activation settings and training set.
mlp_sig5 = MLPClassifier(activation='logistic', alpha=1e-6, 
                         hidden_layer_sizes=(100, 100))
mlp_sig5.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=1e-06, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [42]:
mlp_sig5.score(X_train, y_train)

0.7478360957642726

In [43]:
# 5-fold CV
scores = cross_val_score(mlp_sig5, X_train, y_train, cv=5)
print('Scores: ', scores)

print('\nAccuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

Scores:  [0.73636991 0.72582873 0.73342541 0.7381994  0.72432059]

Accuracy: 0.73 (+/- 0.01)


In [44]:
# Establish and fit the model, with logistic activation settings and training set.
mlp_sig6 = MLPClassifier(activation='logistic', alpha=1e-6, 
                         hidden_layer_sizes=(100, 100, 100))
mlp_sig6.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=1e-06, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [45]:
mlp_sig6.score(X_train, y_train)

0.7445211786372007

In [46]:
# 5-fold CV
scores = cross_val_score(mlp_sig6, X_train, y_train, cv=5)
print('Scores: ', scores)

print('\nAccuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

Scores:  [0.7347596  0.72859116 0.73618785 0.73014046 0.72685398]

Accuracy: 0.73 (+/- 0.01)


In [47]:
# Establish and fit the model, with logistic activation settings and training set.
mlp_sig7 = MLPClassifier(activation='logistic', alpha=1e-6, 
                         hidden_layer_sizes=(100, 100, 100, 100))
mlp_sig7.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=1e-06, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100, 100, 100), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [48]:
mlp_sig7.score(X_train, y_train)

0.719659300184162

In [49]:
# 5-fold CV
scores = cross_val_score(mlp_sig7, X_train, y_train, cv=5)
print('Scores: ', scores)

print('\nAccuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

Scores:  [0.69588222 0.70142726 0.72145488 0.70527285 0.74481806]

Accuracy: 0.71 (+/- 0.04)


The accuracy score decreased with increasing number of layers, and the level of overfitting increased as well, represented by higher standard errors of the cross-validation.

### Number of Neurons per Layer

In [50]:
mlp_sig8 = MLPClassifier(activation='logistic', alpha=1e-6, 
                         hidden_layer_sizes=(1000, 1000))
mlp_sig8.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=1e-06, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(1000, 1000), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [51]:
mlp_sig8.score(X_train, y_train)

0.775598526703499

In [52]:
# 5-fold CV
scores = cross_val_score(mlp_sig8, X_train, y_train, cv=5)
print('Scores: ', scores)

print('\nAccuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

Scores:  [0.74764205 0.73273481 0.73618785 0.77849413 0.770152  ]

Accuracy: 0.75 (+/- 0.04)


Although increasing the number of neurons per layer does improve overall score accuracy more than any other adjustment, the rate of overfitting is still relatively high, and the runtime of this classifier was on the scale of 10+ hours, making it not practical to use for large datasets.